# **Instalações e Bibliotecas**

In [ ]:
%%capture

!pip install shap
!pip install optuna

In [ ]:
import pandas as pd
import numpy as np

from sklearn.impute import KNNImputer

# **Funções**

O arquivo 'functions_pred_cruzada' contém as funções que serão utilizadas para selecionar, pré-processar e treinar e validar os modelos de machine learning

In [ ]:
!gdown 1uVoArwRTJmvbyFLyFniwFvWGHMfiNIWO --quiet

from functions_pred_cruzada import *

# **Dados**

In [4]:
# Dados brutos com todos os tipos de câncer
!gdown 1P19f3kA5_s4sv5Pg6D6UKwwQvZvaAnQT --quiet

df_geral = pd.read_csv('pacigeral.csv')
print(df_geral.shape)
df_geral.head(3)

Columns (35,36,73,74,75,78,89,90) have mixed types. Specify dtype option on import or set low_memory=False.


(1134043, 100)


,ESCOLARI,IDADE,SEXO,UFNASC,UFRESID,IBGE,CIDADE,CATEATEND,DTCONSULT,CLINICA,...,REC03,REC04,IBGEATEN,CIDO,DSCCIDO,HABILIT,HABIT11,HABILIT1,HABILIT2,CIDADEH
0,2,100,2,SP,SP,3550308,SAO PAULO,9,2000-06-09,3,...,NaN,NaN,3550308,80703,CARCINOMA ESCAMOCELULAR SOE,7,CACON com Servio de Oncologia Pedi trica,3,2,SÆo Paulo
1,1,102,1,MG,MG,3133402,ITAPAGIPE,9,2002-04-29,24,...,NaN,NaN,3505500,80903,CARCINOMA BASOCELULAR SOE,7,CACON com Servio de Oncologia Pedi trica,3,2,Barretos
2,2,101,2,MG,SP,3543402,RIBEIRAO PRETO,9,2000-12-03,10,...,NaN,NaN,3543402,80703,CARCINOMA ESCAMOCELULAR SOE,7,CACON com Servio de Oncologia Pedi trica,3,2,RibeirÆo Preto


**Tipos com maior incidência**

In [5]:
df_geral.TOPOGRUP.value_counts(normalize=True).head(15)

C44    0.229273
C50    0.136342
C61    0.103496
C34    0.047272
C53    0.046387
C18    0.037844
C42    0.037599
C16    0.036821
C20    0.028988
C73    0.023747
C77    0.018756
C15    0.018305
C67    0.017882
C32    0.015979
C64    0.015855
Name: TOPOGRUP, dtype: float64

*   C44 - Pele 22,93%
*   C50 - Mama 13,63%
*   C61 - Próstata 10,35%
*   C34 - Pulmão 4,73%
*   C53 - Colo de Útero 4,64%
*   C18 - Cólon 3,78%
*   C42 - ?? 3,76%
*   C16 - Estômago 3,68%
*   C20 - Reto 2,90%
*   C73 - Tiróide 2,37%
*   C77 - ?? 1,88%
*   C15 - Esôfago 1,83%
*   C67 - Bexiga 1,79%
*   C32 - Laringe 1,60%
*   C64 - Rim 1,59%



## **Mama + Próstata**

Seleção dos dados de câncer de mama

In [6]:
# Mama - C50 e Próstata - C61
mama_prost = ['C50', 'C61']

df_mama_prost = data_prep(df_geral, mama_prost)
print(df_mama_prost.shape)
df_mama_prost.head(3)

(211444, 30)


,IDADE,SEXO,IBGE,CATEATEND,DIAGPREV,TOPO,TOPOGRUP,EC,TRATHOSP,NENHUM,...,DIAGTRAT,ANODIAG,DRS,RRAS,IBGEATEN,HABILIT2,ULTIDIAG,ESCOLARI_preench,IBGE_idem_IBGEATEN,presenca_rec
7,102,2,3550308,9,1,C509,C50,IIIB,D,0,...,2,2001,01,6,3550308,2,436,2.0,1,1
39363,28,2,3547809,9,2,C500,C50,I,G,0,...,2,2002,01,1,3547809,1,7107,5.0,1,0
39364,27,2,3512001,2,2,C500,C50,I,H,0,...,0,2003,05,13,3505500,2,6587,4.0,0,1


In [7]:
df_mama_prost.columns

Index(['IDADE', 'SEXO', 'IBGE', 'CATEATEND', 'DIAGPREV', 'TOPO', 'TOPOGRUP',
       'EC', 'TRATHOSP', 'NENHUM', 'CIRURGIA', 'RADIO', 'QUIMIO', 'HORMONIO',
       'TMO', 'IMUNO', 'OUTROS', 'ULTINFO', 'CONSDIAG', 'TRATCONS', 'DIAGTRAT',
       'ANODIAG', 'DRS', 'RRAS', 'IBGEATEN', 'HABILIT2', 'ULTIDIAG',
       'ESCOLARI_preench', 'IBGE_idem_IBGEATEN', 'presenca_rec'],
      dtype='object')

In [8]:
df_mama_prost.TOPOGRUP.unique()

array(['C50', 'C61'], dtype=object)

Criação da coluna de saída

In [9]:
df_mama_prost = get_labels(df_mama_prost)
df_mama_prost.shape

(211444, 32)

In [10]:
df_mama_prost.columns

Index(['IDADE', 'SEXO', 'IBGE', 'CATEATEND', 'DIAGPREV', 'TOPO', 'TOPOGRUP',
       'EC', 'TRATHOSP', 'NENHUM', 'CIRURGIA', 'RADIO', 'QUIMIO', 'HORMONIO',
       'TMO', 'IMUNO', 'OUTROS', 'CONSDIAG', 'TRATCONS', 'DIAGTRAT', 'ANODIAG',
       'DRS', 'RRAS', 'IBGEATEN', 'HABILIT2', 'ESCOLARI_preench',
       'IBGE_idem_IBGEATEN', 'presenca_rec', 'obito_geral', 'sobrevida_ano1',
       'sobrevida_ano3', 'sobrevida_ano5'],
      dtype='object')

Salvando os dados no Drive

In [12]:
df_mama_prost.to_csv('/content/drive/MyDrive/Trabalho/Cancer/Predição cruzada/Datasets/mama_prostata.csv',
                     encoding='utf-8', index=False)